In [3]:
import pandas as pd
import os
import spacy
from collections import Counter
import en_core_web_sm
import geonamescache
nlp = en_core_web_sm.load()
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="AidData")

ModuleNotFoundError: No module named 'geonamescache'

In [2]:
df = pd.read_csv(r'node.csv')
NOI="Thailand"
NOI_CC="TH"

In [3]:
df=df[df['Recipient'].isin([NOI])]

In [4]:
tokens = nlp(''.join(str(df.Description.tolist())))

In [5]:
gc = geonamescache.GeonamesCache()

# gets nested dictionary for countries
countries = gc.get_countries()

# gets nested dictionary for cities
cities = gc.get_cities()

In [6]:
def gen_dict_extract(var, key):
    if isinstance(var, dict):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, (dict, list)):
                yield from gen_dict_extract(v, key)
    elif isinstance(var, list):
        for d in var:
            yield from gen_dict_extract(d, key)

In [7]:
cities = [*gen_dict_extract(cities, 'name')]
countries = [*gen_dict_extract(countries, 'name')]

In [8]:
location_list = []
for ent in tokens.ents:
    if ent.label_ == 'GPE':
        if ent.text in cities:
            location_list.append(ent.text)
        elif ent.text in countries:
            pass
        elif "U.S." in ent.text:
            pass
        else:
            location_list.append(ent.text)
location_counts = Counter(location_list).most_common()
df_location = pd.DataFrame(location_counts, columns =['location', 'count'])

In [9]:
df_map = pd.DataFrame(columns=['location','count','lat','long'])
#df_map = pd.DataFrame()
#df_map

In [10]:
for i, row in df_location.iterrows():
    try:
        lst = df_location.loc[i, :].values.flatten().tolist()
        location = [ lst[0] ]
        location_text = geolocator.geocode(location, country_codes=NOI_CC)    
        lat = location_text.latitude
        long = location_text.longitude
        lst.append(lat)
        lst.append(long)
        #print(lst)
        df_map.loc[len(df_map)] = lst     
    except:
        pass
df_map  


,location,count,lat,long
0,Bangkok,20,13.752544,100.493473
1,Chaiyaphum Province,5,16.272035,101.715382
2,Phuket Island,4,7.741373,98.365467
3,Rayong,3,12.794372,101.370534
4,Chiang Mai,3,18.788278,98.985880
5,Songkhla,2,6.879022,100.549854
6,Phuket,2,7.936602,98.352929
7,the Kingdom of Thailand,1,8.142179,98.597901
8,Tokyo,1,7.848104,98.296390
9,开泰银行,1,13.286720,100.927069


In [11]:
if os.path.isfile('heatmapper_all.csv'):
    print("Append")
else:
    with open('heatmapper_all.csv', 'a') as file:
        file.write('location, count, lat, long\n')    

Append


In [12]:
df_map.to_csv('%s-heatmapper.csv' %NOI_CC, header=True, index=False, mode='w')
df_map.to_csv('heatmapper_all.csv', header=False, index=False, mode='a')